In [ ]:
import torch as t
import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sn
import cfgrib
import pygrib
import os
import time
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import netCDF4 as nc
import datetime as dt
from datetime import datetime

In [ ]:
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
device = t.device('cuda' if t.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# File paths
file_path_train_a = "../nc/accum.nc"
file_path_train_i = "../nc/instant.nc"
file_path_train_p = "../nc/pressure.nc"

In [ ]:
def explore_netcdf(file_path):
    """
    Explore the contents of a netCDF file and return basic information.
    
    Args:
        file_path: Path to the netCDF file
        
    Returns:
        A dictionary containing information about the file
    """
    print(f"\nExploring: {os.path.basename(file_path)}")
    print("="*50)
    
    # Open the file
    dataset = nc.Dataset(file_path, 'r')
    
    # Get dimensions
    dimensions = {dim: len(dataset.dimensions[dim]) for dim in dataset.dimensions}
    print("Dimensions:")
    for dim_name, dim_size in dimensions.items():
        print(f"  {dim_name}: {dim_size}")
    
    # Get variables
    variables = {}
    print("\nVariables:")
    for var_name in dataset.variables:
        var = dataset.variables[var_name]
        var_shape = var.shape
        var_dims = var.dimensions
        var_dtype = var.dtype
        variables[var_name] = {
            'shape': var_shape,
            'dimensions': var_dims,
            'dtype': var_dtype
        }
        
        # Add attributes if available
        if hasattr(var, 'units'):
            variables[var_name]['units'] = var.units
        if hasattr(var, 'long_name'):
            variables[var_name]['long_name'] = var.long_name
            
        print(f"  {var_name}: shape={var_shape}, dims={var_dims}, type={var_dtype}")
        
        # Print some attributes if available
        if hasattr(var, 'units'):
            print(f"    units: {var.units}")
        if hasattr(var, 'long_name'):
            print(f"    long_name: {var.long_name}")
    
    # Get global attributes
    global_attrs = {attr: getattr(dataset, attr) for attr in dataset.ncattrs()}
    print("\nGlobal Attributes:")
    for attr_name, attr_value in global_attrs.items():
        print(f"  {attr_name}: {attr_value}")
    
    # Close the dataset
    dataset.close()
    
    return {
        'dimensions': dimensions,
        'variables': variables,
        'global_attributes': global_attrs
    }

In [ ]:
print(explore_netcdf(file_path_train_a))

In [ ]:
print(explore_netcdf(file_path_train_i))

In [ ]:
print(explore_netcdf(file_path_train_p))

# Our Variables
| Variable | Shape               | Dimensions                                       | Data Type | Units       | Long Name                                      | Type          |
|----------|---------------------|--------------------------------------------------|-----------|-------------|------------------------------------------------|---------------|
| tp       | (2160, 101, 237)    | (valid_time, latitude, longitude)                | float32   | m           | Total precipitation                            | accumulated   |
| slhf     | (2160, 101, 237)    | (valid_time, latitude, longitude)                | float32   | J m**-2     | Time-integrated surface latent heat net flux   | accumulated   |
| sshf     | (2160, 101, 237)    | (valid_time, latitude, longitude)                | float32   | J m**-2     | Time-integrated surface sensible heat net flux | accumulated   |
| ssrd     | (2160, 101, 237)    | (valid_time, latitude, longitude)                | float32   | J m**-2     | Surface short-wave (solar) radiation downwards | accumulated   |
| strd     | (2160, 101, 237)    | (valid_time, latitude, longitude)                | float32   | J m**-2     | Surface long-wave (thermal) radiation downwards| accumulated   |
| u10      | (2160, 101, 237)    | (valid_time, latitude, longitude)                | float32   | m s**-1     | 10 metre U wind component                      | instantaneous |
| v10      | (2160, 101, 237)    | (valid_time, latitude, longitude)                | float32   | m s**-1     | 10 metre V wind component                      | instantaneous |
| d2m      | (2160, 101, 237)    | (valid_time, latitude, longitude)                | float32   | K           | 2 metre dewpoint temperature                   | instantaneous |
| t2m      | (2160, 101, 237)    | (valid_time, latitude, longitude)                | float32   | K           | 2 metre temperature                            | instantaneous |
| sp       | (2160, 101, 237)    | (valid_time, latitude, longitude)                | float32   | Pa          | Surface pressure                               | instantaneous |
| tcc      | (2160, 101, 237)    | (valid_time, latitude, longitude)                | float32   | (0 - 1)     | Total cloud cover                              | instantaneous |
| stl1     | (2160, 101, 237)    | (valid_time, latitude, longitude)                | float32   | K           | Soil temperature level 1                       | instantaneous |
| blh      | (2160, 101, 237)    | (valid_time, latitude, longitude)                | float32   | m           | Boundary layer height                          | instantaneous |
| q        | (2160, 1, 101, 237) | (valid_time, pressure_level, latitude, longitude)| float32   | kg kg**-1   | Specific humidity                              | pressure      |
| t        | (2160, 1, 101, 237) | (valid_time, pressure_level, latitude, longitude)| float32   | K           | Temperature                                    | pressure      |

# Visual Exploration

## Time Series Analysis

### All Time Steps

We begin by just taking a look at the distribution of our variables at a few chosen locations

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import os

# Define file paths for all datasets
file_path_train_i = "f:\\weather_forecasting\\notebooks\\final project\\nc\\instant.nc"
file_path_train_a = "f:\\weather_forecasting\\notebooks\\final project\\nc\\accum.nc"
file_path_train_p = "f:\\weather_forecasting\\notebooks\\final project\\nc\\pressure.nc"

# Define feature sets for all datasets with their dataset names
datasets = {
    'instant': {
        'file_path': file_path_train_i,
        'features': ['t2m', 'd2m', 'stl1', 'sp', 'u10', 'v10', 'tcc', 'blh']
    },
    'accum': {
        'file_path': file_path_train_a,
        'features': ['tp', 'slhf', 'sshf', 'ssrd', 'strd']
    },
    'pressure': {
        'file_path': file_path_train_p,
        'features': ['t', 'q']
    }
}

# Define cities with their coordinates (latitude, longitude)
cities = [
    ("Los Angeles", 34.05, -118.24),
    ("San Francisco", 37.77, -122.42),
    ("Seattle", 47.61, -122.33),
    ("Missoula", 46.87, -113.99),
    ("Salt Lake City", 40.76, -111.89),
    ("Denver", 39.74, -104.99),
    ("Phoenix", 33.45, -112.07),
    ("San Antonio", 29.42, -98.49),
    ("New Orleans", 29.95, -90.07),
    ("Kansas City", 39.10, -94.58),
    ("Chicago", 41.88, -87.63),
    ("New York City", 40.71, -74.01),
    ("Boston", 42.36, -71.06),
    ("Philadelphia", 39.95, -75.17),
    ("Atlanta", 33.75, -84.39),
    ("Miami", 25.76, -80.19)
]

# Directory to save images (current working directory)
image_dir = "."

# Ensure the output directory exists
os.makedirs(image_dir, exist_ok=True)

# Function to extract time series data at a specific location
def extract_time_series(xrds, var_name, dataset_name, lat, lon):
    # If the dataset is 'pressure', select the 850 hPa level
    if dataset_name == 'pressure':
        dims_to_exclude = {'latitude', 'longitude', 'valid_time'}
        pressure_dim = [dim for dim in xrds[var_name].dims if dim not in dims_to_exclude]
        if pressure_dim:
            pressure_dim = pressure_dim[0]
            try:
                xrds = xrds.sel(**{pressure_dim: 850})
            except KeyError:
                xrds = xrds.isel(**{pressure_dim: 0})

    # Interpolate the variable at the given latitude and longitude
    data = xrds[var_name].interp(latitude=lat, longitude=lon, method='nearest')

    # Get the time steps and values
    time_steps = range(len(data.valid_time))
    values = data.values

    return time_steps, values

# Function to create a 4x4 subplot figure for a variable across all cities
def create_subplot_time_series_plot(var_name, dataset_name, time_series_data):
    # Create a 4x4 subplot grid
    fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(20, 20))
    axes = axes.flatten()

    # Get long name and units for the variable
    xrds = xr.open_dataset(datasets[dataset_name]['file_path'], engine="netcdf4")
    long_name = xrds[var_name].attrs.get('long_name', var_name)
    units = xrds[var_name].attrs.get('units', '')
    xrds.close()

    # Plot time series for each city in its subplot
    for idx, ((city, lat, lon), (time_steps, values)) in enumerate(zip(cities, time_series_data)):
        ax = axes[idx]
        ax.plot(time_steps, values, color='g')  # Green line, no markers
        ax.set_title(city, fontsize=10)
        ax.set_xlabel('Time Step', fontsize=8)
        ax.set_ylabel(f"{long_name} ({units})", fontsize=8)
        ax.grid(True)
        ax.tick_params(axis='both', labelsize=8)

    # Adjust layout to minimize white space
    plt.tight_layout(pad=2.0, w_pad=1.0, h_pad=1.0)

    # Add a super title for the entire figure
    fig.suptitle(f"Time Series of {long_name} Across Cities", fontsize=16, y=1.02)

    # Save the plot
    output_path = os.path.join(image_dir, f"time_series_subplots_{var_name}.png")
    plt.savefig(output_path, bbox_inches="tight", dpi=300)
    print(f"Time series subplots for {var_name} saved as {output_path}")
    plt.show()
    plt.close(fig)

# Process each dataset and variable
variables = []
for dataset_name, info in datasets.items():
    for var_name in info['features']:
        variables.append((var_name, dataset_name))

# Generate time series subplots for each variable
for var_name, dataset_name in variables:
    print(f"\nProcessing variable {var_name} from {dataset_name} dataset...\n")

    # Load the dataset
    xrds = xr.open_dataset(datasets[dataset_name]['file_path'], engine="netcdf4")

    # Extract time series data for each city
    time_series_data = []
    for city, lat, lon in cities:
        time_steps, values = extract_time_series(xrds, var_name, dataset_name, lat, lon)
        time_series_data.append((time_steps, values))

    xrds.close()

    # Create the 4x4 subplot figure
    create_subplot_time_series_plot(var_name, dataset_name, time_series_data)

### First Day & First Week

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import os

# Define file paths for all datasets
file_path_train_i = "f:\\weather_forecasting\\notebooks\\final project\\nc\\instant.nc"
file_path_train_a = "f:\\weather_forecasting\\notebooks\\final project\\nc\\accum.nc"
file_path_train_p = "f:\\weather_forecasting\\notebooks\\final project\\nc\\pressure.nc"

# Define feature sets for all datasets with their dataset names
datasets = {
    'instant': {
        'file_path': file_path_train_i,
        'features': ['t2m', 'd2m', 'stl1', 'sp', 'u10', 'v10', 'tcc', 'blh']
    },
    'accum': {
        'file_path': file_path_train_a,
        'features': ['tp', 'slhf', 'sshf', 'ssrd', 'strd']
    },
    'pressure': {
        'file_path': file_path_train_p,
        'features': ['t', 'q']
    }
}

# Define cities with their coordinates (latitude, longitude)
cities = [
    ("Los Angeles", 34.05, -118.24),
    ("San Francisco", 37.77, -122.42),
    ("Seattle", 47.61, -122.33),
    ("Missoula", 46.87, -113.99),
    ("Salt Lake City", 40.76, -111.89),
    ("Denver", 39.74, -104.99),
    ("Phoenix", 33.45, -112.07),
    ("San Antonio", 29.42, -98.49),
    ("New Orleans", 29.95, -90.07),
    ("Kansas City", 39.10, -94.58),
    ("Chicago", 41.88, -87.63),
    ("New York City", 40.71, -74.01),
    ("Boston", 42.36, -71.06),
    ("Philadelphia", 39.95, -75.17),
    ("Atlanta", 33.75, -84.39),
    ("Miami", 25.76, -80.19)
]

# Directory to save images (current working directory)
image_dir = "."

# Ensure the output directory exists
os.makedirs(image_dir, exist_ok=True)

# Function to extract time series data at a specific location
def extract_time_series(xrds, var_name, dataset_name, lat, lon, max_timesteps):
    # If the dataset is 'pressure', select the 850 hPa level
    if dataset_name == 'pressure':
        dims_to_exclude = {'latitude', 'longitude', 'valid_time'}
        pressure_dim = [dim for dim in xrds[var_name].dims if dim not in dims_to_exclude]
        if pressure_dim:
            pressure_dim = pressure_dim[0]
            try:
                xrds = xrds.sel(**{pressure_dim: 850})
            except KeyError:
                xrds = xrds.isel(**{pressure_dim: 0})

    # Limit the data to the specified number of time steps
    data = xrds[var_name].isel(valid_time=slice(0, max_timesteps))

    # Interpolate the variable at the given latitude and longitude
    data = data.interp(latitude=lat, longitude=lon, method='nearest')

    # Get the time steps and values
    time_steps = range(len(data.valid_time))
    values = data.values

    return time_steps, values

# Function to create a 4x4 subplot figure for a variable across all cities
def create_subplot_time_series_plot(var_name, dataset_name, time_series_data, max_timesteps, time_range_label):
    # Create a 4x4 subplot grid
    fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(20, 20))
    axes = axes.flatten()

    # Get long name and units for the variable
    xrds = xr.open_dataset(datasets[dataset_name]['file_path'], engine="netcdf4")
    long_name = xrds[var_name].attrs.get('long_name', var_name)
    units = xrds[var_name].attrs.get('units', '')
    xrds.close()

    # Plot time series for each city in its subplot
    for idx, ((city, lat, lon), (time_steps, values)) in enumerate(zip(cities, time_series_data)):
        ax = axes[idx]
        ax.plot(time_steps, values, color='g')  # Green line, no markers
        ax.set_title(city, fontsize=10)
        ax.set_xlabel('Time Step', fontsize=8)
        ax.set_ylabel(f"{long_name} ({units})", fontsize=8)
        ax.grid(True)
        ax.tick_params(axis='both', labelsize=8)

    # Adjust layout to minimize white space
    plt.tight_layout(pad=2.0, w_pad=1.0, h_pad=1.0)

    # Add a super title for the entire figure
    fig.suptitle(f"Time Series of {long_name} Across Cities ({time_range_label})", fontsize=16, y=1.02)

    # Save the plot
    output_path = os.path.join(image_dir, f"time_series_subplots_{var_name}_{max_timesteps}timesteps.png")
    plt.savefig(output_path, bbox_inches="tight", dpi=300)
    print(f"Time series subplots for {var_name} ({time_range_label}) saved as {output_path}")
    plt.show()
    plt.close(fig)

# Process each dataset and variable for a specific number of time steps
def process_time_series_plots(max_timesteps, time_range_label):
    variables = []
    for dataset_name, info in datasets.items():
        for var_name in info['features']:
            variables.append((var_name, dataset_name))

    # Generate time series subplots for each variable
    for var_name, dataset_name in variables:
        print(f"\nProcessing variable {var_name} from {dataset_name} dataset ({time_range_label})...\n")

        # Load the dataset
        xrds = xr.open_dataset(datasets[dataset_name]['file_path'], engine="netcdf4")

        # Extract time series data for each city
        time_series_data = []
        for city, lat, lon in cities:
            time_steps, values = extract_time_series(xrds, var_name, dataset_name, lat, lon, max_timesteps)
            time_series_data.append((time_steps, values))

        xrds.close()

        # Create the 4x4 subplot figure
        create_subplot_time_series_plot(var_name, dataset_name, time_series_data, max_timesteps, time_range_label)

# Run for 1 day (first 24 time steps)
process_time_series_plots(max_timesteps=24, time_range_label="1 Day")

# Run for 1 week (first 168 time steps)
process_time_series_plots(max_timesteps=168, time_range_label="1 Week")

## Spatial Grid Analysis

With some time series analysis under our belt we next want to a get a grasp of what the data actually looks like across the entire grid at on specific moment in time

In [ ]:
# Define file paths for all datasets
file_path_train_i = "f:\\weather_forecasting\\notebooks\\final project\\nc\\instant.nc"
file_path_train_a = "f:\\weather_forecasting\\notebooks\\final project\\nc\\accum.nc"
file_path_train_p = "f:\\weather_forecasting\\notebooks\\final project\\nc\\pressure.nc"

# Define feature sets for all datasets
feature_sets = {
    'instant': ['t2m', 'd2m', 'stl1', 'sp', 'u10', 'v10', 'tcc', 'blh'],
    'accum': ['tp', 'slhf', 'sshf', 'ssrd', 'strd'],
    'pressure': ['t', 'q']
}

# Define colormaps for variables
colormaps = {
    # instant dataset
    't2m': 'coolwarm',
    'd2m': 'coolwarm',
    'stl1': 'coolwarm',
    'sp': 'coolwarm',
    'u10': 'terrain',
    'v10': 'terrain',
    'tcc': 'terrain',
    'blh': 'terrain',
    # accum dataset
    'tp': 'coolwarm',
    'slhf': 'coolwarm',
    'sshf': 'coolwarm',
    'ssrd': 'terrain',
    'strd': 'terrain',
    # pressure dataset
    't': 'coolwarm',
    'q': 'terrain'
}

# Time index to plot
time_index = 0

# Directory to save images (current working directory)
image_dir = "."

# Ensure the output directory exists
os.makedirs(image_dir, exist_ok=True)

# Function to generate and save individual plots
def generate_individual_plots(file_path, dataset_name, features):
    try:
        # Load NetCDF file
        xrds = xr.open_dataset(file_path, engine="netcdf4")
        print(f"Successfully opened {dataset_name} dataset: {file_path}")

        # Loop over each variable in the feature set
        for var_name in features:
            try:
                if var_name in xrds.variables:
                    # Select data for the current variable and time step
                    data_at_time = xrds[var_name].isel(valid_time=time_index)

                    # Print the dimensions and shape of the data
                    print(f"{var_name} dimensions: {data_at_time.dims}")
                    print(f"{var_name} data shape before selections: {data_at_time.shape}")

                    # If the dataset is 'pressure', select the 850 hPa level
                    if dataset_name == 'pressure':
                        # Identify the pressure level dimension
                        dims_to_exclude = {'latitude', 'longitude', 'valid_time'}
                        pressure_dim = [dim for dim in data_at_time.dims if dim not in dims_to_exclude]
                        
                        if pressure_dim:
                            pressure_dim = pressure_dim[0]
                            print(f"Found pressure dimension '{pressure_dim}' for {var_name}")
                            # Print available pressure levels
                            print(f"Available {pressure_dim} values: {xrds[pressure_dim].values}")
                            # Select the 850 hPa level
                            try:
                                data_at_time = data_at_time.sel(**{pressure_dim: 850})
                                print(f"Selected 850 hPa level for {var_name} using dimension {pressure_dim}")
                            except KeyError:
                                print(f"Error: 850 hPa not found in {pressure_dim} for {var_name}. Using first level instead.")
                                data_at_time = data_at_time.isel(**{pressure_dim: 0})
                        else:
                            print(f"Warning: No pressure level dimension found for {var_name} in pressure dataset")

                    # Print the shape of the data after selections
                    print(f"{var_name} data shape after selections: {data_at_time.shape}")

                    # Get long name and units from variable attributes
                    long_name = xrds[var_name].attrs.get('long_name', var_name)
                    units = xrds[var_name].attrs.get('units', '')
                    title = f"{long_name} ({units})" if units else long_name

                    # Check the data's latitude and longitude bounds
                    lon = data_at_time.longitude
                    lat = data_at_time.latitude
                    lon_min, lon_max = float(lon.min()), float(lon.max())
                    lat_min, lat_max = float(lat.min()), float(lat.max())
                    print(f"{var_name} bounds: lon [{lon_min}, {lon_max}], lat [{lat_min}, {lat_max}]")

                    # Create figure with Cartopy projection
                    fig = plt.figure(figsize=(14, 8))
                    ax = plt.axes(projection=ccrs.PlateCarree())

                    # Create meshgrid for pcolormesh
                    lon, lat = np.meshgrid(data_at_time.longitude, data_at_time.latitude)

                    # Plot the data using pcolormesh
                    im = ax.pcolormesh(
                        lon, lat, data_at_time,
                        cmap=colormaps[var_name],
                        transform=ccrs.PlateCarree()
                    )

                    # Add continental US features
                    ax.add_feature(cfeature.LAND, edgecolor='black')
                    ax.add_feature(cfeature.COASTLINE)
                    ax.add_feature(cfeature.BORDERS)
                    ax.add_feature(cfeature.STATES, edgecolor='gray')

                    # Set extent to match the data's bounds, with a very small buffer
                    buffer = 0.1
                    ax.set_extent(
                        [lon_min - buffer, lon_max + buffer, lat_min - buffer, lat_max + buffer],
                        crs=ccrs.PlateCarree()
                    )

                    # Force the axes to fill the entire figure (leave space for colorbar)
                    ax.set_position([0, 0, 0.85, 1])

                    # Add a manually positioned colorbar
                    cbar = plt.colorbar(
                        im, 
                        ax=ax, 
                        orientation='vertical',
                        fraction=0.03,
                        pad=0.01,
                        shrink=0.95,
                        anchor=(1, 0.5),
                        aspect=30
                    )
                    cbar.set_label('')
                    cbar.ax.tick_params(labelsize=12)

                    # Add title with long name and units
                    plt.title(title, pad=20, fontsize=16)

                    # Remove all figure margins
                    fig.set_tight_layout(True)
                    fig.subplots_adjust(left=0, right=1, top=1, bottom=0)

                    # Save the plot at higher DPI
                    output_path = os.path.join(image_dir, f"{dataset_name}_{var_name}_time{time_index}_plot.png")
                    plt.savefig(output_path, bbox_inches=None, dpi=400)
                    plt.close(fig)
                    print(f"Successfully plotted and saved {var_name} for {dataset_name}")
                else:
                    print(f"Warning: Variable {var_name} not found in {dataset_name} dataset.")
            except Exception as e:
                print(f"Error plotting {var_name} for {dataset_name}: {str(e)}")
                plt.close()
        return xrds
    except Exception as e:
        print(f"Error opening {dataset_name} dataset {file_path}: {str(e)}")
        return None

# Function to combine plots into a figure
def combine_plots(dataset_name, features, nrows, ncols, figsize, suffix):
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize)
    axes = axes.flatten()

    plot_idx = 0
    for var_name in features:
        image_path = os.path.join(image_dir, f"{dataset_name}_{var_name}_time{time_index}_plot.png")
        try:
            if os.path.exists(image_path):
                img = imread(image_path)
                axes[plot_idx].imshow(img)
                axes[plot_idx].axis('off')
                plot_idx += 1
            else:
                print(f"Warning: Image not found: {image_path}")
        except Exception as e:
            print(f"Error loading {image_path}: {str(e)}")

    # Turn off unused subplots (if any)
    for i in range(plot_idx, len(axes)):
        axes[i].axis('off')

    # Adjust layout to minimize white space
    plt.tight_layout(pad=1.0, w_pad=0.5, h_pad=0.5)

    # Save and display the figure
    combined_output_path = os.path.join(image_dir, f"combined_weather_plots_{dataset_name}_{suffix}.png")
    plt.savefig(combined_output_path, bbox_inches="tight", dpi=300)
    print(f"Combined figure for {dataset_name} saved as {combined_output_path}")
    plt.show()
    plt.close(fig)

# Process all datasets
# Dataset 1: instant.nc (two 2x2 figures)
xrds_i = generate_individual_plots(file_path_train_i, "instant", feature_sets['instant'])
if xrds_i is not None:
    # First 2x2 figure for the first four variables
    combine_plots("instant", feature_sets['instant'][:4], nrows=2, ncols=2, figsize=(28, 16), suffix="1")
    # Second 2x2 figure for the last four variables
    combine_plots("instant", feature_sets['instant'][4:], nrows=2, ncols=2, figsize=(28, 16), suffix="2")
    xrds_i.close()

# Dataset 2: accum.nc (5x1 figure)
xrds_a = generate_individual_plots(file_path_train_a, "accum", feature_sets['accum'])
if xrds_a is not None:
    combine_plots("accum", feature_sets['accum'], nrows=5, ncols=1, figsize=(14, 40), suffix="all")
    xrds_a.close()

# Dataset 3: pressure.nc (2x1 figure)
xrds_p = generate_individual_plots(file_path_train_p, "pressure", feature_sets['pressure'])
if xrds_p is not None:
    combine_plots("pressure", feature_sets['pressure'], nrows=2, ncols=1, figsize=(14, 16), suffix="all")
    xrds_p.close()

Let get a better look at what an entire day looks like (in 6 hour increments)

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib.image import imread
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import os
import numpy as np

# Define file paths for all datasets
file_path_train_i = "f:\\weather_forecasting\\notebooks\\final project\\nc\\instant.nc"
file_path_train_a = "f:\\weather_forecasting\\notebooks\\final project\\nc\\accum.nc"
file_path_train_p = "f:\\weather_forecasting\\notebooks\\final project\\nc\\pressure.nc"

# Define feature sets for all datasets with their dataset names
datasets = {
    'instant': {
        'file_path': file_path_train_i,
        'features': ['t2m', 'd2m', 'stl1', 'sp', 'u10', 'v10', 'tcc', 'blh']
    },
    'accum': {
        'file_path': file_path_train_a,
        'features': ['tp', 'slhf', 'sshf', 'ssrd', 'strd']
    },
    'pressure': {
        'file_path': file_path_train_p,
        'features': ['t', 'q']
    }
}

# Define colormaps for variables
colormaps = {
    't2m': 'coolwarm',
    'd2m': 'coolwarm',
    'stl1': 'coolwarm',
    'sp': 'coolwarm',
    'u10': 'terrain',
    'v10': 'terrain',
    'tcc': 'terrain',
    'blh': 'terrain',
    'tp': 'coolwarm',
    'slhf': 'coolwarm',
    'sshf': 'coolwarm',
    'ssrd': 'terrain',
    'strd': 'terrain',
    't': 'coolwarm',
    'q': 'terrain'
}

# Time indices to plot (1st, 6th, 12th, 18th time steps, zero-based)
time_indices = [0, 5, 11, 17]

# Directory to save images (current working directory)
image_dir = "."

# Ensure the output directory exists
os.makedirs(image_dir, exist_ok=True)

# Function to generate and save individual plots
def generate_individual_plots(file_path, dataset_name, features, time_idx):
    # Load NetCDF file
    xrds = xr.open_dataset(file_path, engine="netcdf4")

    # Loop over each variable in the feature set
    for var_name in features:
        # Select data for the current variable and time step
        data_at_time = xrds[var_name].isel(valid_time=time_idx)

        # If the dataset is 'pressure', select the 850 hPa level
        if dataset_name == 'pressure':
            dims_to_exclude = {'latitude', 'longitude', 'valid_time'}
            pressure_dim = [dim for dim in data_at_time.dims if dim not in dims_to_exclude]
            if pressure_dim:
                pressure_dim = pressure_dim[0]
                try:
                    data_at_time = data_at_time.sel(**{pressure_dim: 850})
                except KeyError:
                    data_at_time = data_at_time.isel(**{pressure_dim: 0})

        # Get long name and units from variable attributes
        long_name = xrds[var_name].attrs.get('long_name', var_name)
        units = xrds[var_name].attrs.get('units', '')
        title = f"{long_name} ({units}) at Time Step {time_idx + 1}"

        # Create figure with Cartopy projection
        fig = plt.figure(figsize=(14, 8))
        ax = plt.axes(projection=ccrs.PlateCarree())

        # Create meshgrid for pcolormesh
        lon, lat = np.meshgrid(data_at_time.longitude, data_at_time.latitude)

        # Plot the data using pcolormesh
        im = ax.pcolormesh(
            lon, lat, data_at_time,
            cmap=colormaps[var_name],
            transform=ccrs.PlateCarree()
        )

        # Add continental US features
        ax.add_feature(cfeature.LAND, edgecolor='black')
        ax.add_feature(cfeature.COASTLINE)
        ax.add_feature(cfeature.BORDERS)
        ax.add_feature(cfeature.STATES, edgecolor='gray')

        # Set extent to match the data's bounds, with a very small buffer
        buffer = 0.1
        lon_min, lon_max = float(data_at_time.longitude.min()), float(data_at_time.longitude.max())
        lat_min, lat_max = float(data_at_time.latitude.min()), float(data_at_time.latitude.max())
        ax.set_extent(
            [lon_min - buffer, lon_max + buffer, lat_min - buffer, lat_max + buffer],
            crs=ccrs.PlateCarree()
        )

        # Force the axes to fill the entire figure (leave space for colorbar)
        ax.set_position([0, 0, 0.85, 1])

        # Add a manually positioned colorbar
        cbar = plt.colorbar(
            im, 
            ax=ax, 
            orientation='vertical',
            fraction=0.03,
            pad=0.01,
            shrink=0.95,
            anchor=(1, 0.5),
            aspect=30
        )
        cbar.set_label('')
        cbar.ax.tick_params(labelsize=12)

        # Add title with long name, units, and time step
        plt.title(title, pad=20, fontsize=16)

        # Remove all figure margins
        fig.set_tight_layout(True)
        fig.subplots_adjust(left=0, right=1, top=1, bottom=0)

        # Save the plot at higher DPI
        output_path = os.path.join(image_dir, f"{dataset_name}_{var_name}_time{time_idx}_plot.png")
        plt.savefig(output_path, bbox_inches=None, dpi=400)
        plt.close(fig)

    return xrds

# Function to create 2x2 figures for each variable across all time steps
def create_variable_comparison_plots(variables):
    for var_name, dataset_name in variables:
        print(f"\nCreating comparison plot for {var_name} across all time steps...\n")
        fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(28, 16))
        axes = axes.flatten()

        for idx, time_idx in enumerate(time_indices):
            image_path = os.path.join(image_dir, f"{dataset_name}_{var_name}_time{time_idx}_plot.png")
            if os.path.exists(image_path):
                img = imread(image_path)
                axes[idx].imshow(img)
                axes[idx].axis('off')
            else:
                print(f"Warning: Image not found: {image_path}")

        # Adjust layout to minimize white space
        plt.tight_layout(pad=1.0, w_pad=0.5, h_pad=0.5)

        # Save and display the figure
        combined_output_path = os.path.join(image_dir, f"variable_comparison_{var_name}_across_timesteps.png")
        plt.savefig(combined_output_path, bbox_inches="tight", dpi=300)
        print(f"Combined figure for {var_name} saved as {combined_output_path}")
        plt.show()
        plt.close(fig)

# Process all datasets for each time step to generate individual plots
for time_idx in time_indices:
    print(f"\nProcessing time step {time_idx + 1} (index {time_idx})...\n")

    # Dataset 1: instant.nc
    xrds_i = generate_individual_plots(datasets['instant']['file_path'], "instant", datasets['instant']['features'], time_idx)
    if xrds_i is not None:
        xrds_i.close()

    # Dataset 2: accum.nc
    xrds_a = generate_individual_plots(datasets['accum']['file_path'], "accum", datasets['accum']['features'], time_idx)
    if xrds_a is not None:
        xrds_a.close()

    # Dataset 3: pressure.nc
    xrds_p = generate_individual_plots(datasets['pressure']['file_path'], "pressure", datasets['pressure']['features'], time_idx)
    if xrds_p is not None:
        xrds_p.close()

# Create 2x2 figures for each variable across all time steps
variables = []
for dataset_name, info in datasets.items():
    for var_name in info['features']:
        variables.append((var_name, dataset_name))

create_variable_comparison_plots(variables)

# Data Processing

With a better understanding our data set we turn our attention towards data processing and creating our tensors. The expver is a key used to identify the specific model version and will not be of importance to us. After dropping this from our dataset we merge the three datasets, perform some basic conversions from Kelvin to Fahrenheit, and add the temperature spread variable t_spread_f. The new spread variable has two purposes: the first is to act as a basic measure of the relative humidity. It will not be perfect given the actual equation: RH = 100 * (exp((17.625 * d2m) / (243.04 + d2m)) / exp((17.625 * t2m) / (243.04 + t2m))), but it will help. The second purpose of introducing this new variable is to replace the dewpoint tempertaure feature d2m altogether to avoid any collinearity concerns which arise from using both the air temperature and dewpoint temperature to train our model. We also drop all of the original temperature variables (in Kelvin) and save the new dataset. With the goal being to train a model that will predict the temperature at the next hour we construct our input tensor using all time steps up to the last and our target tensor using all time steps but the first. In this way the model will use time step t to predict time step t+1. To achieve this we convert each time step into a numpy array and then a pytorch tensor object. Lastly, we normalize our tensors and save them for easy access when constructing our model.

In [ ]:
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
device = t.device('cuda' if t.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# File paths
file_path_train_a = "../nc/accum.nc"
file_path_train_i = "../nc/instant.nc"
file_path_train_p = "../nc/pressure.nc"

# Open datasets with larger chunks
# Training Data (January - March 2025)
start = time.time()
ds_a_train = xr.open_dataset(file_path_train_a, chunks={'valid_time': 100})
ds_i_train = xr.open_dataset(file_path_train_i, chunks={'valid_time': 100})
ds_p_train = xr.open_dataset(file_path_train_p, chunks={'valid_time':100})
# Drop expver if present
if 'expver' in ds_a_train.coords:
    ds_a_train = ds_a_train.drop_vars('expver')
if 'expver' in ds_i_train.coords:
    ds_i_train = ds_i_train.drop_vars('expver')
if 'expver' in ds_p_train.coords:
    ds_p_train = ds_p_train.drop_vars('expver')
print(f"Training datasets opened in {time.time() - start:.2f}s: ds_i_train shape {ds_i_train.dims}, ds_a_train shape {ds_a_train.dims}, ds_p_train shape {ds_p_train.dims}")

# Define variable names
i_vars = ['t2m', 'd2m', 'tcc', 'sp', 'u10', 'v10', 'stl1', 'blh']
a_vars = ['tp', 'sshf', 'slhf', 'ssrd', 'strd']
p_vars = ['t', 'q']
print(f"Variables defined: instant {i_vars}, accum {a_vars}, pressure {p_vars}")

# Merge datasets
# Training Data
start = time.time()
ds_train_merge = xr.merge([ds_i_train[i_vars], ds_a_train[a_vars], ds_p_train[p_vars]])
print(f"Training datasets merged in {time.time() - start:.2f}s: shape {ds_train_merge.dims}, dtype {ds_train_merge[i_vars[0]].dtype}")

# Convert to Fahrenheit
def kelvin_to_fahrenheit(temp_k):
    temp_f = (temp_k - 273.15) * 9/5 + 32
    return temp_f

# Converting to fahrenheit & adding temperature spread
start = time.time()
ds_train_merge = ds_train_merge.assign(t2m_f=lambda ds: kelvin_to_fahrenheit(ds['t2m']))
ds_train_merge = ds_train_merge.assign(d2m_f=lambda ds: kelvin_to_fahrenheit(ds['d2m']))
ds_train_merge = ds_train_merge.assign(t_f=lambda ds: kelvin_to_fahrenheit(ds['t']))
ds_train_merge = ds_train_merge.assign(t_spread_f=lambda ds: ds['t2m_f']-ds['d2m_f'])
print(f"Converting to F in {time.time() - start:.2f}")

# Dropping the original temperature variables as well as d2m_f
ds_train_merge = ds_train_merge.drop_vars(['t2m', 'd2m', 'd2m_f', 't'])

# Update variables
i_vars = ['t2m_f', 't_spread_f', 'tcc', 'sp', 'u10', 'v10', 'stl1', 'blh']
a_vars = ['tp', 'sshf', 'slhf', 'ssrd', 'strd']
p_vars = ['t_f', 'q']
all_vars = a_vars+i_vars+p_vars

# Save intermediates
start = time.time()
ds_train_merge.to_netcdf('f:/weather_forecasting/ds_train.nc')
print(f"Saved intermediates in {time.time() - start:.2f}s: train shape {ds_train_merge.dims}")

# Stack spatial dimensions
start = time.time()
ds_train_stacked = ds_train_merge[all_vars].stack(node=('latitude', 'longitude'))
print(f"Stacked spatial dims in {time.time() - start:.2f}s: train shape {ds_train_stacked.dims}")

# Create inputs and targets (all time steps)
start = time.time()
# Training
total_train_time_steps = ds_train_stacked.dims['valid_time']
print(f"Total training time steps: {total_train_time_steps}")
train_inputs = ds_train_stacked.isel(valid_time=slice(0, total_train_time_steps - 1))
train_targets = ds_train_stacked.isel(valid_time=slice(1, total_train_time_steps))

# Tensor build
# Convert to numpy array
start = time.time()
train_inputs_array = train_inputs.to_array().values.transpose(1, 2, 0, 3)
train_targets_array = train_targets.to_array().values.transpose(1, 2, 0, 3)
print(f"Converted to arrays in {time.time() - start:.2f}s: train inputs shape {train_inputs_array.shape}, train targets shape {train_targets_array.shape}, dtype {train_inputs_array.dtype}")

# Convert to pytorch tensors
start = time.time()
train_inputs_tensor = t.tensor(train_inputs_array, dtype=t.float32).to(device)
train_targets_tensor = t.tensor(train_targets_array, dtype=t.float32).to(device)
print(f"Tensors built in {time.time() - start:.2f}s: train inputs shape {train_inputs_tensor.shape}, train targets shape {train_targets_tensor.shape}, device {train_inputs_tensor.device}")

# Normalize tensors (using mean and std from the training dataset only)
start = time.time()
train_inputs_mean = train_inputs_tensor.mean(dim=(0, 1), keepdim=True)
train_inputs_std = train_inputs_tensor.std(dim=(0, 1), keepdim=True)
train_targets_mean = train_targets_tensor.mean(dim=(0, 1), keepdim=True)
train_targets_std = train_targets_tensor.std(dim=(0, 1), keepdim=True)

# Normalize training tensors
train_inputs_tensor = (train_inputs_tensor - train_inputs_mean) / (train_inputs_std + 1e-8)
train_targets_tensor = (train_targets_tensor - train_targets_mean) / (train_targets_std + 1e-8)

# Save normalized tensors
start = time.time()
t.save(train_inputs_tensor.cpu(), 'f:/weather_forecasting/notebooks/final project/tensors/train_inputs_norm.pt')
t.save(train_targets_tensor.cpu(), 'f:/weather_forecasting/notebooks/final project/tensors/train_targets_norm.pt')
print(f"Normalized tensors saved in {time.time() - start:.2f}s to f:/weather_forecasting/notebooks/final project/tensors")